## 요약



In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 그래프의 폰트 출력을 선명하게 (svg, retina 등이 있음)
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("default")
# 이미지가 레이아웃 안으로 들어오도록 함
# https://matplotlib.org/stable/users/explain/axes/constrainedlayout_guide.html
mpl.rcParams.update({"figure.constrained_layout.use": True})

#font, line, marker 등의 배율 설정: paper, notebook, talk, poster
sns.set_context("paper") 
#배색 설정: tab10, Set2, Accent, husl
sns.set_palette("Set2") 
#눈금, 배경, 격자 설정: ticks, white, whitegrid, dark, darkgrid
# withegrid: 눈금을 그리고, 각 축의 눈금을 제거
sns.set_style("whitegrid") 

In [3]:
# 로컬에서 

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [4]:
# 필요한 패키지 import
from bs4 import BeautifulSoup
import requests
import re
import os, natsort

In [5]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_usedbook'
save_dir = 'processed'
date = 240711
file_name = f'unused_filtered_{date}.csv'

### 앞선 내용 요약
- 비중고도서 목록과 중고도서 목록, 2 파일을 관리하기로 결정
    - 비중고도서 ItemId를 기준으로 각 중고정보 페이지를 접근하여 크롤링
        - 비중고도서 페이지 url
            - ```https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={}```
        - 중고정보 페이지 url
            -```https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId={}&TabType=3&Fix=1``` 
        - ItemId에 규칙이 따로 없는 것으로 추정
    - 비중고도서 목록에는 각 도서별 정보 (ItemId, 도서명, 저자, 출판사, 세일즈포인트 등)
    - 중고도서 목록에는 각 매물 별 정보 (책이름, ItemId, 중고등급, 판매가, 할인율)
- 비중고도서 목록
    - 구분 : '국내도서' 중 저자, 출판사, 대분류가 null이 아닌 경우만 추려서 사용
    - ItemId 기준 중복 정리
    - ItemId 기준 keep=last로 해서 중복 제거
    - 부정확할 수 있는 정보는 이후 크롤링해서 체크 후 업데이트 하는 것으로
        - 저자가 복수일 때 양식이 통일되지 않음
            - 다음 두 양식이 혼용
            1. AA, BB, CC 지음
            2. AA 외 지음
        - SalesPoint는 이후에는 crawling해서 확인하는 걸로
            - 매일 업데이트 되기 때문에 도서별 정보가 다소 다를 수 있음

**정해진 프로세스**

0. 임의의 연/월/주를 10개 골라서 itemid 목록 작성
1. 임의의 새 상품 페이지를 들어간다
2. 새 상품 관련 정보 수집
3. 해당 도서의 중고 정보 페이지로 접근
4. 각 도서별 중고 상품 정보 수집

In [6]:
file_path = os.path.join(PRJCT_PATH,save_dir,file_name)
df_unused = pd.read_csv(file_path)
df_unused.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9328 entries, 0 to 9327
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   순번/순위       9328 non-null   float64
 1   구분          9328 non-null   object 
 2   상품명         9328 non-null   object 
 3   ItemId      9328 non-null   int64  
 4   ISBN13      9264 non-null   object 
 5   부가기호        7886 non-null   object 
 6   저자/아티스트     9328 non-null   object 
 7   출판사/제작사     9328 non-null   object 
 8   출간일         9328 non-null   object 
 9   정가          9328 non-null   int64  
 10  판매가         9328 non-null   int64  
 11  마일리지        9328 non-null   object 
 12  세일즈포인트      9328 non-null   float64
 13  대표분류(대분류명)  9328 non-null   object 
 14  source      9328 non-null   object 
dtypes: float64(2), int64(3), object(10)
memory usage: 1.1+ MB


In [7]:
# 가져올 책 범위,ID.. 값 정의
id_list = list(df_unused.ItemId.values)

### 중고정보 페이지 구조
- ``#Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody`` 이하에 중고 매물가 있음 내용이 있음
- 이하 tr 들이 row
- 첫번째 tr은 header, 두번째 이하로 도서 매물 내용
- 만약 도서 매물이 없는 경우, tr은 header에 해당하는 하나 밖에 없음

In [27]:
## researching

url_usedinfo = 'https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId={}&TabType=3&Fix=1'
book_id = 254468327
url = url_usedinfo.format(book_id) 
data = dict()
errored_item=dict()
base_table ='#Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table' 

r = requests.get(url)
if r.status_code != 200:
    errored_item[book_id] = 'status!=200'
    raise Exception
html=r.text
soup=BeautifulSoup(html, 'lxml') #  BeautifulSoup 클래스의 인스턴스 생성
table = soup.select_one(base_table)

used_list = table.find_all('tr')
if len(used_list) <= 1 : raise Exception

In [ ]:
len(table.find_all('tr'))

3

In [36]:
display(used_list[1].select_one('td:nth-child(3) > span > span').get_text())
display(used_list[1].select_one('td:nth-child(4) > div > ul > li.Ere_sub_pink > span').get_text())
display(used_list[1].select_one('td:nth-child(4) > div > ul > li:nth-child(3)').get_text())

# 등급, 판매가, 배송비, url

#url : #Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody > tr:nth-child(3) > td.sell_tableCF1 > a
# #Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody > tr:nth-child(5) > td.sell_tableCF1 > a
#등급 : #Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody > tr:nth-child(3) > td:nth-child(3) > span > span 
#판매가 : #Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody > tr:nth-child(3) > td:nth-child(4) > div > ul > li.Ere_sub_pink > span
#배송비 : #Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table > tbody > tr:nth-child(3) > td:nth-child(4) > div > ul > li:nth-child(3)

'최상'

'5,600'

'배송비 : 2,500원'

In [41]:
list(filter(lambda x : 'dict' in x,used_list[2].select_one('td.sell_tableCF1 > a').__dir__()))

['__dict__']

In [51]:
used_list[2].select_one('td:nth-child(3) > span > span').__dict__['contents']

['최상']

In [52]:
used_list[2].select_one('td:nth-child(4) > div > ul > li:nth-child(3)').__dict__['contents']

['배송비 : 2,500원']

In [48]:
used_list[2].select_one('td.sell_tableCF1 > a').__dict__['attrs']['href']

'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=340672167'

In [56]:
selector_dict = {
   'quality': ('td:nth-child(3) > span > span',lambda x : x.__dict__['contents'][0].strip()),
   'price': ('td:nth-child(4) > div > ul > li.Ere_sub_pink > span',lambda x : x.get_text().strip().replace(',','')),
   'delivery fee': ('td:nth-child(4) > div > ul > li:nth-child(3)',lambda x : x.get_text().strip().split(' : ')[1][:-1].replace(',','')),
   'url': ('td.sell_tableCF1 > a',lambda x : x['href']),
}

data_dict = dict()
for i in range(1,len(used_list)):
   content = used_list[i]
   rslt = {
      key : func(content.select_one(selector))
      for key,(selector,func) in selector_dict.items()
   }
   data_dict[i] = rslt
   
data_dict

{1: {'quality': '최상',
  'price': '5600',
  'delivery fee': '2500',
  'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=341867477'},
 2: {'quality': '최상',
  'price': '5600',
  'delivery fee': '2500',
  'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=340672167'}}

#### 정리된 코드

In [58]:
def class_name(clss):
  name = str(type(clss)).strip()
  name = name[1:-1].split(' ')
  return name[1]

class CustomError(Exception):
  def __init__(self,msg):
    super().__init__(msg)

In [57]:
from tqdm import tqdm
from functools import partial

tqdm_1line = partial(tqdm,position=0,leave=True)

In [76]:
import time, random

# ID를 증가시키며 책 정보 크롤링
url_usedinfo = 'https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId={}&TabType=3&Fix=1'

base_table ='#Ere_prod_allwrap_box > div.Ere_prod_middlewrap > div.Ere_usedsell_table > table' 
selector_dict = {
   'quality': ('td:nth-child(3) > span > span',lambda x : x.__dict__['contents'][0].strip()),
   'price': ('td:nth-child(4) > div > ul > li.Ere_sub_pink > span',lambda x : x.get_text().strip().replace(',','')),
   'delivery fee': ('td:nth-child(4) > div > ul > li:nth-child(3)',lambda x : x.get_text().strip().split(' : ')[1][:-1].replace(',','')),
   'url': ('td.sell_tableCF1 > a',lambda x : x['href']),
}

data_dict = dict()
errored_item = dict()
rest_count = 0
work_limit = 100
null_used = list()

for n,book_id in enumerate(tqdm(id_list[:work_limit])):
    url = url_usedinfo.format(book_id) 
    data = dict()
    if n % (100+random.uniform(0,5)) == 1 :
        rest_count,sleeping_time = rest_count+1, (random.uniform(1,3))/2
        print('time to rest **^^** : ',rest_count," | ",sleeping_time)
        time.sleep(sleeping_time)
    try:
        r = requests.get(url)
        if r.status_code != 200: raise Exception('bad request')
        html=r.text
        soup=BeautifulSoup(html, 'lxml') #  BeautifulSoup 클래스의 인스턴스 생성
        table = soup.select_one(base_table)

        used_list = table.find_all('tr')
        if len(used_list) <= 1 : null_used.append(book_id)
        data,error_count = dict(), 0
        for i in range(1,len(used_list)):
            content = used_list[i]
            try :
                data[i] = {
                   key : func(content.select_one(selector))
                   for key,(selector,func) in selector_dict.items()
                }
            except: error_count += 1
        if data : data_dict[book_id] = data
        elif len(used_list) > 1 : raise Exception('all product in table raised error')
        if error_count : raise Exception('some selector raised error')
    except Exception as e:
        errored_item[book_id] = f'{class_name(e)}/{e}'

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<00:36,  2.70it/s]

time to rest **^^** :  1  |  1.00355880063622


100%|██████████| 100/100 [01:00<00:00,  1.66it/s]


In [77]:
len(errored_item), len(data_dict)

(1, 80)

In [80]:
errored_item

{173075: "'AttributeError'/'NoneType' object has no attribute 'find_all'"}

**에러**
- 173075 : 19금 도서이기 때문에 로그인이 필요했음
    - "화성남자 금성여자의 침실 가꾸기" /https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId=173075